In [1]:
from selenium import webdriver
import pandas as pd
from scrap_article_current_tab import scrap_article

from datetime import datetime as dt
from datetime import timedelta as td
from os import makedirs
from os.path import exists
from time import sleep
from json import loads

In [2]:
now = dt.now()
year = now.year
month = now.month
day = now.day
ts = str(int(round(now.timestamp())))

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
chrome = webdriver.Chrome(options=options)

In [4]:
# chrome = webdriver.Chrome()

In [5]:
chrome.get(url = 'http://wyborcza.pl/0,75248.html?str=100_23719317')

In [6]:
sleep(10)

In [7]:
# try to click rodo (first click opens sometimes an ad also)
for i in range(3):
    try:
        sleep(3)
        chrome.find_element_by_xpath('//*[@id="rodoNotificationWrapper"]/div[1]/div/p[2]')\
            .click()
    except:
        pass

In [8]:
# close new window if exists
try:
    chrome.switch_to.window(chrome.window_handles[1])
    chrome.close()
except:
    pass

In [9]:
chrome.switch_to.window(chrome.window_handles[0])

In [10]:
chrome.get('https://login.wyborcza.pl/')

In [11]:
chrome\
    .find_element_by_xpath('//*[@id="wyborczaEmail"]')\
    .send_keys('slusarczyk1@wp.pl')

In [12]:
chrome\
    .find_element_by_xpath('//*[@id="wyborczaPassword"]')\
    .send_keys('Sraniejebanko1')

In [13]:
chrome\
    .find_element_by_xpath('/html/body/section/section[1]/form/div[4]/button')\
    .click()

In [14]:
sleep(3)

In [15]:
# http://wyborcza.pl/0,93566.html?str=2_23721352 for video

In [16]:
urls = []
for i in range(1651, 2000):
    chrome.get(url = 'http://wyborcza.pl/0,75248.html?str='+str(i)+'_23719317')
    for entry in chrome\
        .find_element_by_xpath('//*[@id="pagetype_index"]/section[5]/div/main/div/div[2]/div/div[2]/ul')\
        .find_elements_by_class_name('entry'):
        
        url = entry.find_element_by_tag_name('h3')\
                .find_element_by_tag_name('a')\
                .get_attribute('href')
        #scrap_article(chrome, url, first_time)
        urls.append(url)
    break

In [17]:
import pandas as pd

In [18]:
# df = pd.DataFrame(urls, columns = ['urls'])

In [19]:
# df.to_csv('aaa.csv')

In [20]:
df = pd.read_csv('44k_urls.csv')

In [21]:
errors = []

In [25]:
for i, url in enumerate(df['urls'].iloc[::-1]):
    print(url)
    try:
        a = scrap_article(chrome, url, True)
        with open('comments/'+a['url'].split('/')[-1].split(',')[2], 'w') as f:
            f.write(str({'row': {
               'comments': a,
               'timestamp': dt.timestamp(dt.now())
               }
            }))
        print(a['url'].split('/')[-1].split(',')[2])

    except:
        errors.append(url)
        print(url)
        
    

http://wyborcza.pl/7,75399,24810476,legalizacja-malzenstw-osob-jednej-plci-na-tajwanie-zaostrzyla.html
24810476
http://wyborcza.pl/7,75398,24811352,duda-wygrywa-z-tuskiem.html
24811352
http://wyborcza.pl/7,75398,24811329,teoria-spisku-w-pis-podejrzewaja-ze-ziobro-mogl-inspirowac.html


KeyboardInterrupt: 

In [23]:
pd.DataFrame(errors, columns = ['urls']).to_csv('error_urls.csv')